<a href="https://colab.research.google.com/github/Miliyas/Generative_AI/blob/main/Attention_model_mib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import torch
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch.nn as nn
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
lr = 0.0005
vec_len = 50
seq_len = 20
num_epochs = 50
label_col = "Product"
tokens_path = "Output/tokens.pkl"
labels_path = "Output/labels.pkl"
data_path = "/content/drive/MyDrive/complaints_new.csv"
model_path = "Output/attention.pth"
vocabulary_path = "Output/vocabulary.pkl"
embeddings_path = "Output/embeddings.pkl"
glove_vector_path = "/content/drive/MyDrive/glove.6B.50d.txt"
text_col_name = "Consumer complaint narrative"
label_encoder_path = "Output/label_encoder.pkl"
product_map = {'Vehicle loan or lease': 'vehicle_loan',
               'Credit reporting, credit repair services, or other personal consumer reports': 'credit_report',
               'Credit card or prepaid card': 'card',
               'Money transfer, virtual currency, or money service': 'money_transfer',
               'virtual currency': 'money_transfer',
               'Mortgage': 'mortgage',
               'Payday loan, title loan, or personal loan': 'loan',
               'Debt collection': 'debt_collection',
               'Checking or savings account': 'savings_account',
               'Credit card': 'card',
               'Bank account or service': 'savings_account',
               'Credit reporting': 'credit_report',
               'Prepaid card': 'card',
               'Payday loan': 'loan',
               'Other financial service': 'others',
               'Virtual currency': 'money_transfer',
               'Student loan': 'loan',
               'Consumer Loan': 'loan',
               'Money transfers': 'money_transfer'}

In [4]:
mkdir Output

In [5]:
def save_file(name, obj):
    """
    Function to save an object as pickle file
    """
    with open(name, 'wb') as f:
        pickle.dump(obj, f)


def load_file(name):
    """
    Function to load a pickle object
    """
    return pickle.load(open(name, "rb"))

# **Different word embedding techniques**

**word2vec**

In [ ]:
pip install gensim


In [ ]:
import gensim.downloader as api

# Download the pre-trained model
model = api.load("word2vec-google-news-300")

# Save the model to a file
model.save("word2vec-google-news-300.model")


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
import gensim

# Load a pre-trained Word2Vec model
from gensim.models import KeyedVectors

# Load the model
model = KeyedVectors.load("word2vec-google-news-300.model", mmap='r')

# Get the vector for a word
word_vector = model['computer']

# Find similar words
similar_words = model.most_similar('computer')

# Print the results
print(f"Word: 'computer', Vector: {word_vector}")
print(f"Similar words: {similar_words}")

Word: 'computer', Vector: [ 1.07421875e-01 -2.01171875e-01  1.23046875e-01  2.11914062e-01
 -9.13085938e-02  2.16796875e-01 -1.31835938e-01  8.30078125e-02
  2.02148438e-01  4.78515625e-02  3.66210938e-02 -2.45361328e-02
  2.39257812e-02 -1.60156250e-01 -2.61230469e-02  9.71679688e-02
 -6.34765625e-02  1.84570312e-01  1.70898438e-01 -1.63085938e-01
 -1.09375000e-01  1.49414062e-01 -4.65393066e-04  9.61914062e-02
  1.68945312e-01  2.60925293e-03  8.93554688e-02  6.49414062e-02
  3.56445312e-02 -6.93359375e-02 -1.46484375e-01 -1.21093750e-01
 -2.27539062e-01  2.45361328e-02 -1.24511719e-01 -3.18359375e-01
 -2.20703125e-01  1.30859375e-01  3.66210938e-02 -3.63769531e-02
 -1.13281250e-01  1.95312500e-01  9.76562500e-02  1.26953125e-01
  6.59179688e-02  6.93359375e-02  1.02539062e-02  1.75781250e-01
 -1.68945312e-01  1.21307373e-03 -2.98828125e-01 -1.15234375e-01
  5.66406250e-02 -1.77734375e-01 -2.08984375e-01  1.76757812e-01
  2.38037109e-02 -2.57812500e-01 -4.46777344e-02  1.88476562e-01

In [ ]:
similar_words = model.most_similar('king')
print(similar_words)


[('kings', 0.7138045430183411), ('queen', 0.6510956883430481), ('monarch', 0.6413194537162781), ('crown_prince', 0.6204220056533813), ('prince', 0.6159993410110474), ('sultan', 0.5864824056625366), ('ruler', 0.5797567367553711), ('princes', 0.5646552443504333), ('Prince_Paras', 0.5432944297790527), ('throne', 0.5422105193138123)]


**GloVe**

## Process glove embeddings
---

In [6]:
with open(glove_vector_path, "rt") as f:
    emb = f.readlines()

In [7]:
vocabulary, embeddings = [], []

for item in emb:
    vocabulary.append(item.split()[0])
    embeddings.append(item.split()[1:])

In [8]:
#convert list into numpy array
embeddings = np.array(embeddings, dtype=np.float32)

In [9]:
vocabulary = ["<pad>", "<unk>"] + vocabulary

In [10]:
embeddings = np.vstack([np.ones(50, dtype=np.float32),
                        np.mean(embeddings, axis=0),
                        embeddings])

In [11]:
embeddings

array([[ 1.        ,  1.        ,  1.        , ...,  1.        ,
         1.        ,  1.        ],
       [-0.12920076, -0.28866628, -0.01224866, ...,  0.10069408,
         0.00653003,  0.01685157],
       [ 0.418     ,  0.24968   , -0.41242   , ..., -0.18411   ,
        -0.11514   , -0.78581   ],
       ...,
       [-0.51181   ,  0.058706  ,  1.0913    , ..., -0.25003   ,
        -1.125     ,  1.5863    ],
       [-0.75898   , -0.47426   ,  0.4737    , ...,  0.78954   ,
        -0.014116  ,  0.6448    ],
       [ 0.072617  , -0.51393   ,  0.4728    , ..., -0.18907   ,
        -0.59021   ,  0.55559   ]], dtype=float32)

In [12]:
embeddings.shape

(400002, 50)

In [13]:
save_file(embeddings_path, embeddings)
save_file(vocabulary_path, vocabulary)

## Process text data
---

In [14]:
data = pd.read_csv(data_path)

In [15]:
data.head()

,Product,Consumer complaint narrative
0,Vehicle loan or lease,I contacted Ally on Friday XX/XX/XXXX after fa...
1,"Credit reporting, credit repair services, or o...",Hello This complaint is against the three cred...
2,"Credit reporting, credit repair services, or o...",I am a victim of Identity Theft & currently ha...
3,"Credit reporting, credit repair services, or o...",Two accounts are still on my credit history af...
4,"Credit reporting, credit repair services, or o...",Receiving daily telephone call ( s ) from XXXX...


In [16]:
data.dropna(subset=[text_col_name], inplace=True)

In [17]:
data.replace({label_col: product_map}, inplace=True)

### Encode labels

**LabelEncoder**

In [18]:
['apple', 'banana', 'orange', 'apple', 'banana']
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
encoded_fruits = le.fit_transform(['apple', 'banana', 'orange', 'apple', 'banana'])

print(encoded_fruits)

[0 1 2 0 1]


**OneHotEncoder:**


In [19]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
encoded_fruits = ohe.fit_transform(np.array(['apple', 'banana', 'orange', 'apple', 'banana']).reshape(-1, 1))

print(encoded_fruits)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


**LabelEncoder**

In [20]:
label_encoder = LabelEncoder()
label_encoder.fit(data[label_col])
label_encoder.classes_

array(['card', 'credit_report', 'debt_collection', 'loan',
       'money_transfer', 'mortgage', 'others', 'savings_account',
       'vehicle_loan'], dtype=object)

In [21]:
labels = label_encoder.transform(data[label_col])

In [22]:
labels

array([8, 1, 1, ..., 5, 1, 1])

In [23]:
list(label_encoder.inverse_transform([8, 1, 1,5]))

['vehicle_loan', 'credit_report', 'credit_report', 'mortgage']

In [24]:
save_file(labels_path, labels)
save_file(label_encoder_path, label_encoder)

### Process the text column

In [25]:
type(data[text_col_name])

pandas.core.series.Series

In [26]:
input_text = list(data[text_col_name])

In [27]:
len(input_text)

809343

### Convert text to lower case

In [28]:
input_text = [i.lower() for i in tqdm(input_text)]

100%|██████████| 809343/809343 [00:01<00:00, 696058.20it/s]


### Remove punctuations except apostrophe

In [29]:
input_text = [re.sub(r"[^\w\d'\s]+", " ", i)
              for i in tqdm(input_text)]

100%|██████████| 809343/809343 [00:24<00:00, 32458.83it/s]


### Remove digits

In [30]:
input_text = [re.sub("\d+", "", i) for i in tqdm(input_text)]

100%|██████████| 809343/809343 [00:15<00:00, 52309.52it/s]


### Remove more than one consecutive instance of 'x'

In [31]:
input_text = [re.sub(r'[x]{2,}', "", i) for i in tqdm(input_text)]

100%|██████████| 809343/809343 [00:13<00:00, 61349.06it/s]


### Remove multiple spaces with single space

In [32]:
input_text = [re.sub(' +', ' ', i) for i in tqdm(input_text)]

100%|██████████| 809343/809343 [00:40<00:00, 20032.65it/s]


### Tokenize the text

In [33]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [34]:
tokens = [word_tokenize(t) for t in tqdm(input_text)]

100%|██████████| 809343/809343 [06:54<00:00, 1952.27it/s]


### Take the first 20 tokens in each complaint text

In [35]:
tokens[0][:20]

['i',
 'contacted',
 'ally',
 'on',
 'friday',
 'after',
 'falling',
 'behind',
 'on',
 'payments',
 'due',
 'to',
 'being',
 'out',
 'of',
 'work',
 'for',
 'a',
 'short',
 'period']

In [36]:
tokens = [i[:20] if len(i) > 19 else ['<pad>'] * (20 - len(i)) + i
          for i in tqdm(tokens)]

100%|██████████| 809343/809343 [00:10<00:00, 80186.81it/s]


### Convert tokens to integer indices from vocabulary

In [37]:
def token_index(tokens, vocabulary, missing='<unk>'):
    """
    :param tokens: List of word tokens
    :param vocabulary: All words in the embeddings
    :param missing: Token for words not present in the vocabulary
    :return: List of integers representing the word tokens
    """
    idx_token = []
    for text in tqdm(tokens):
        idx_text = []
        for token in text:
            if token in vocabulary:
                idx_text.append(vocabulary.index(token))
            else:
                idx_text.append(vocabulary.index(missing))
        idx_token.append(idx_text)
    return idx_token

In [38]:
print(vocabulary.index('credit'))

1166


In [39]:
len(vocabulary)

400002

In [40]:
tokens = token_index(tokens, vocabulary)

100%|██████████| 809343/809343 [1:38:29<00:00, 136.97it/s]


### Save the tokens

In [41]:
save_file(tokens_path, tokens)

## Create attention model
---

In [42]:
class AttentionModel(nn.Module):

    def __init__(self, vec_len, seq_len, n_classes):
        super(AttentionModel, self).__init__()
        self.vec_len = vec_len
        self.seq_len = seq_len
        self.attn_weights = torch.cat([torch.tensor([[0.]]),
                                       torch.randn(vec_len, 1) /
                                       torch.sqrt(torch.tensor(vec_len))])
        self.attn_weights.requires_grad = True
        self.attn_weights = nn.Parameter(self.attn_weights)
        self.activation = nn.Tanh()
        self.softmax = nn.Softmax(dim=1)
        self.linear = nn.Linear(vec_len + 1, n_classes)

    def forward(self, input_data):
        hidden = torch.matmul(input_data, self.attn_weights)
        hidden = self.activation(hidden)
        attn = self.softmax(hidden)
        attn = attn.repeat(1, 1, self.vec_len + 1).reshape(attn.shape[0],
                                                           self.seq_len,
                                                           self.vec_len + 1)
        attn_output = input_data * attn
        attn_output = torch.sum(attn_output, axis=1)
        output = self.linear(attn_output)
        return output

## Create PyTorch dataset
---

In [43]:
class TextDataset(torch.utils.data.Dataset):

    def __init__(self, tokens, embeddings, labels):
        """
        :param tokens: List of word tokens
        :param embeddings: Word embeddings (from glove)
        :param labels: List of labels
        """
        self.tokens = tokens
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        emb = torch.tensor(self.embeddings[self.tokens[idx], :])
        input_ = torch.cat((torch.ones(emb.shape[0],1), emb), dim=1)
        return torch.tensor(self.labels[idx]), input_

### Function to train the model

In [44]:
def train(train_loader, valid_loader, model, criterion, optimizer,
          device, num_epochs, model_path):
    """
    Function to train the model
    :param train_loader: Data loader for train dataset
    :param valid_loader: Data loader for validation dataset
    :param model: Model object
    :param criterion: Loss function
    :param optimizer: Optimizer
    :param device: CUDA or CPU
    :param num_epochs: Number of epochs
    :param model_path: Path to save the model
    """
    best_loss = 1e8
    for i in range(num_epochs):
        print(f"Epoch {i+1} of {num_epochs}")
        valid_loss, train_loss = [], []
        model.train()
        # Train loop
        for batch_labels, batch_data in tqdm(train_loader):
            # Move data to GPU if available
            batch_labels = batch_labels.to(device)
            batch_data = batch_data.to(device)
            # Forward pass
            batch_output = model(batch_data)
            batch_output = torch.squeeze(batch_output)
            # Calculate loss
            loss = criterion(batch_output, batch_labels)
            train_loss.append(loss.item())
            optimizer.zero_grad()
            # Backward pass
            loss.backward()
            # Gradient update step
            optimizer.step()
        model.eval()
        # Validation loop
        for batch_labels, batch_data in tqdm(valid_loader):
            # Move data to GPU if available
            batch_labels = batch_labels.to(device)
            batch_data = batch_data.to(device)
            # Forward pass
            batch_output = model(batch_data)
            batch_output = torch.squeeze(batch_output)
            # Calculate loss
            loss = criterion(batch_output, batch_labels)
            valid_loss.append(loss.item())
        t_loss = np.mean(train_loss)
        v_loss = np.mean(valid_loss)
        print(f"Train Loss: {t_loss}, Validation Loss: {v_loss}")
        if v_loss < best_loss:
            best_loss = v_loss
            # Save model if validation loss improves
            torch.save(model.state_dict(), model_path)
        print(f"Best Validation Loss: {best_loss}")

### Function to test the model

In [45]:
def test(test_loader, model, criterion, device):
    """
    Function to test the model
    :param test_loader: Data loader for test dataset
    :param model: Model object
    :param criterion: Loss function
    :param device: CUDA or CPU
    """
    model.eval()
    test_loss = []
    test_accu = []
    for batch_labels, batch_data in tqdm(test_loader):
        # Move data to device
        batch_labels = batch_labels.to(device)
        batch_data = batch_data.to(device)
        # Forward pass
        batch_output = model(batch_data)
        batch_output = torch.squeeze(batch_output)
        # Calculate loss
        loss = criterion(batch_output, batch_labels)
        test_loss.append(loss.item())
        batch_preds = torch.argmax(batch_output, axis=1)
        # Move predictions to CPU
        if torch.cuda.is_available():
            batch_labels = batch_labels.cpu()
            batch_preds = batch_preds.cpu()
        # Compute accuracy
        test_accu.append(accuracy_score(batch_labels.detach().
                                        numpy(),
                                        batch_preds.detach().
                                        numpy()))
    test_loss = np.mean(test_loss)
    test_accu = np.mean(test_accu)
    print(f"Test Loss: {test_loss}, Test Accuracy: {test_accu}")

## Train attention model
---

### Load the files

In [46]:
tokens = load_file(tokens_path)
labels = load_file(labels_path)
embeddings = load_file(embeddings_path)
label_encoder = load_file(label_encoder_path)
num_classes = len(label_encoder.classes_)
vocabulary = load_file(vocabulary_path)

### Split data into train, validation and test sets

In [47]:
X_train, X_test, y_train, y_test = train_test_split(tokens, labels,
                                                    test_size=0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_train,
                                                      y_train,
                                                      test_size=0.25)

### Create PyTorch datasets

In [48]:
train_dataset = TextDataset(X_train, embeddings, y_train)
valid_dataset = TextDataset(X_valid, embeddings, y_valid)
test_dataset = TextDataset(X_test, embeddings, y_test)

### Create data loaders

In [49]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=16,
                                           shuffle=True,
                                           drop_last=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                                           batch_size=16)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=16)

### Create model object

In [50]:
device = torch.device("cuda:0" if torch.cuda.is_available()
                      else "cpu")

In [52]:
device

device(type='cuda', index=0)

In [51]:
model = AttentionModel(vec_len, seq_len, num_classes)

### Move the model to GPU if available

In [53]:
if torch.cuda.is_available():
    model = model.cuda()

### Define loss function and optimizer

In [54]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

### Training loop

In [55]:
train(train_loader, valid_loader, model, criterion, optimizer,
      device, num_epochs, model_path)

Epoch 1 of 50


100%|██████████| 10117/10117 [00:12<00:00, 796.84it/s]


Train Loss: 1.080838189019501, Validation Loss: 1.0014305693235772
Best Validation Loss: 1.0014305693235772
Epoch 2 of 50


100%|██████████| 10117/10117 [00:12<00:00, 815.79it/s]


Train Loss: 0.9894175603067659, Validation Loss: 0.9844283009778491
Best Validation Loss: 0.9844283009778491
Epoch 3 of 50


...

Train Loss: 0.9545122117039792, Validation Loss: 0.9568092063889739
Best Validation Loss: 0.9568092063889739
Epoch 49 of 50


100%|██████████| 10117/10117 [00:12<00:00, 797.04it/s]


Train Loss: 0.9543354374209582, Validation Loss: 0.956332837391173
Best Validation Loss: 0.956332837391173
Epoch 50 of 50


100%|██████████| 10117/10117 [00:12<00:00, 810.49it/s]

Train Loss: 0.9538907272399355, Validation Loss: 0.9557613654871685
Best Validation Loss: 0.9557613654871685


### Test the model

In [56]:
test(test_loader, model, criterion, device)

100%|██████████| 10117/10117 [00:18<00:00, 536.92it/s]

Test Loss: 0.9557410880527065, Test Accuracy: 0.6720979919556573


## Predict on new text
---

In [57]:
input_text = '''I am a victim of Identity Theft & currently have an Experian account that
I can view my Experian Credit Report and getting notified when there is activity on
my Experian Credit Report. For the past 3 days I've spent a total of approximately 9
hours on the phone with Experian. Every time I call I get transferred repeatedly and
then my last transfer and automated message states to press 1 and leave a message and
someone would call me. Every time I press 1 I get an automatic message stating than you
before I even leave a message and get disconnected. I call Experian again, explain what
is happening and the process begins again with the same end result. I was trying to have
this issue attended and resolved informally but I give up after 9 hours. There are hard
hit inquiries on my Experian Credit Report that are fraud, I didn't authorize, or recall
and I respectfully request that Experian remove the hard hit inquiries immediately just
like they've done in the past when I was able to speak to a live Experian representative
in the United States. The following are the hard hit inquiries : BK OF XXXX XX/XX/XXXX
XXXX XXXX XXXX  XX/XX/XXXX XXXX  XXXX XXXX  XX/XX/XXXX XXXX  XX/XX/XXXX XXXX  XXXX
XX/XX/XXXX'''

### Process input text

In [58]:
input_text = input_text.lower()
input_text = re.sub(r"[^\w\d'\s]+", " ", input_text)
input_text = re.sub("\d+", "", input_text)
input_text = re.sub(r'[x]{2,}', "", input_text)
input_text = re.sub(' +', ' ', input_text)
tokens = word_tokenize(input_text)

In [59]:
tokens = ['<pad>']*(20-len(tokens))+tokens

In [60]:
idx_token = []
for token in tokens:
    if token in vocabulary:
        idx_token.append(vocabulary.index(token))
    else:
        idx_token.append(vocabulary.index('<unk>'))

In [61]:
token_emb = embeddings[idx_token,:]
token_emb = token_emb[:seq_len, :]
inp = torch.from_numpy(token_emb)

In [62]:
inp = torch.cat((torch.ones(inp.shape[0],1), inp), dim=1)

In [63]:
device = torch.device("cuda:0" if torch.cuda.is_available()
                      else "cpu")

In [64]:
inp = inp.to(device)
inp = torch.unsqueeze(inp, 0)

In [65]:
label_encoder = load_file(label_encoder_path)
num_classes = len(label_encoder.classes_)

In [66]:
# Create model object
model = AttentionModel(vec_len, seq_len, num_classes)

# Load trained weights
model.load_state_dict(torch.load(model_path))

# Move the model to GPU if available
if torch.cuda.is_available():
    model = model.cuda()

# Forward pass
out = torch.squeeze(model(inp))

# Find predicted class
prediction = label_encoder.classes_[torch.argmax(out)]
print(f"Predicted  Class: {prediction}")

Predicted  Class: credit_report
